In [ ]:
import os
import json
import pandas as pd
import dtale
import numpy as np

In [ ]:
splits = ["train","dev","test"]
datasets = ["ontonotes","litbank","preco"]
base_data_dir = "../../coref_resources/data/"
model_name = "longformer_speaker"
cross_val_split = "0"

In [ ]:
all_splits = []
for split in splits:
    split_data = []
    for dataset in datasets:
        data_dir = os.path.join(base_data_dir,dataset,model_name)
        print(data_dir)
        if dataset == "litbank":
            data_dir = os.path.join(data_dir,cross_val_split)
        
        jsonl_file = os.path.join(data_dir, f'{split}.4096.jsonlines')
        if jsonl_file is None:
            raise ValueError(f"No relevant files at {data_dir}")
        
        with open(jsonl_file) as f:
            for line in f:
                load_dict = json.loads(line.strip())
                load_dict["dataset_name"] = dataset
                split_data.append(load_dict)
    all_splits.append(split_data)

train_data, dev_data, test_data = all_splits

In [ ]:
mention_counts = []
mention_lengths = []
entity_counts = []

for document in train_data:
    # print(document.keys())
    entity_counts.append(len(document["clusters"]))
    
    mention_count = 0
    for cluster in document["clusters"]:
        mention_count += len(cluster)
    if document["doc_key"] == "pt/nt/47/nt_4711_0":
        print(mention_count)
        print(sum([len(cluster) for cluster in document["clusters"]]))
        print(entity_counts[-1])
    mention_counts.append(mention_count)
    
    for cluster in document["clusters"]:
        for mention in cluster:
            mention_lengths.append(mention[1] - mention[0] + 1)


In [ ]:
entity_counts = np.array(entity_counts)
mention_counts = np.array(mention_counts)
mention_lengths = np.array(mention_lengths)

entity_counts_series = pd.Series(entity_counts)
mention_counts_series = pd.Series(mention_counts)
mention_lengths_series = pd.Series(mention_lengths)

print(entity_counts_series.describe())
print(mention_counts_series.describe())
print(mention_lengths_series.describe())

In [ ]:
mention_counts[mention_counts > 350]

In [ ]:
mention_lengths[mention_lengths > 20].shape[0]/mention_lengths.shape[0]

In [ ]:
mention_counts_series.hist(bins=100)